In [2]:
# =============================================================================
# Setup
# =============================================================================

USE_DRIVE = True
DRIVE_ROOT = "/content/drive/MyDrive" if USE_DRIVE else "H:/My Drive"

if USE_DRIVE:
    import os
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        print("Montando Google Drive...")
        drive.mount('/content/drive')
    else:
        print("Google Drive ya está montado.")


ROOT = os.path.join(DRIVE_ROOT)
ROOT_DIR = os.path.join(ROOT, "Debbuging Neuro")
DATA_DIR = os.path.join(ROOT_DIR, "Data")
REPO_DIR = os.path.join(ROOT, "individualized_prescriptive_inference")

os.chdir(ROOT)

print(f"Working dir: {ROOT_DIR}")

Montando Google Drive...
Mounted at /content/drive
Working dir: /content/drive/MyDrive/Debbuging Neuro


In [3]:
%%capture
!pip install git+https://github.com/NotaCS/Functionnectome.git nibabel tqdm

In [4]:
# =============================================================================
# Paths
# =============================================================================
import os
import glob
import pickle

LESIONS_DIR = os.path.join(REPO_DIR, "lesions")
DISCONN_DIR = os.path.join(REPO_DIR, "disconnectomes")
PROGRESS_FILE = os.path.join(REPO_DIR, "disconn_progress.pkl")
PRIORS_H5 = os.path.join(REPO_DIR, "disconnectomes", "priors_full_proba_3T_comp9_thr0p01.h5")

os.makedirs(DISCONN_DIR, exist_ok=True)

print(f"Lesiones: {LESIONS_DIR}")
print(f"Output: {DISCONN_DIR}")
print(f"Priors existe: {os.path.exists(PRIORS_H5)}")

Lesiones: /content/drive/MyDrive/individualized_prescriptive_inference/lesions
Output: /content/drive/MyDrive/individualized_prescriptive_inference/disconnectomes
Priors existe: True


In [5]:
# =============================================================================
# Importar quickDisco y ver cómo usarlo
# =============================================================================
from Functionnectome import quickDisco
import inspect

# Ver la firma de la función principal
print("Function probaMap_fromMask:")
print(inspect.signature(quickDisco.probaMap_fromMask))
help(quickDisco.probaMap_fromMask)

Function probaMap_fromMask:
(roiFile, priorsLoc, priors_type, outFile=None, proc=1, maxVal=False, templateFile=None)
Help on function probaMap_fromMask in module Functionnectome.quickDisco:

probaMap_fromMask(roiFile, priorsLoc, priors_type, outFile=None, proc=1, maxVal=False, templateFile=None)
    Read the ROI and the associated maps, then create and save the new map

    Parameters
    ----------
    roiFile : str
        Full path to the ROI nifti file.
    priorsLoc : str
        Path to either the HDF5 file or the folder containing all the nifti maps.
    priors_type : str
        Type of priors input, either 'h5' or 'nii'.
    outFile : str
        Full file path for the new map to be created.
    proc : int
        Number of processes used. If > 1, will run parallel processing
    maxVal : bool
        If true, will return the max value of the priors for each voxel. The output will thus be between 0 and 1.
        Otherwise, returns the sum of the priors.
    templateFile : str

In [6]:
# =============================================================================
# Download Priors
# =============================================================================
import os
from Functionnectome import functionnectome as fun

# Directory to save the priors
PRIORS_DIR = os.path.join(REPO_DIR, "disconnectomes")
os.makedirs(PRIORS_DIR, exist_ok=True)  #

# Name of the prior to download
target_prior = 'V2.P.WB - Whole brain, Probabilistic'

# Execute the download process
# This function handles URL requests and unzipping internally
fun.Download_H5(PRIORS_DIR, target_prior)

# The expected downloaded file path:
PRIORS_H5 = os.path.join(PRIORS_DIR, "priors_full_proba_3T_comp9_thr0p01.h5")
print(f"Priors saved at: {PRIORS_H5}")
print(f"File exists: {os.path.exists(PRIORS_H5)}")

Unzipping...
Done
Priors saved at: /content/drive/MyDrive/individualized_prescriptive_inference/disconnectomes/priors_full_proba_3T_comp9_thr0p01.h5
File exists: True


In [6]:
from Functionnectome import quickDisco
import nibabel as nib
import numpy as np

def generate_disconnectome(lesion_path, output_path, priors_h5_path, n_proc=1):
    """
    Generates a disconnectome using the Functionnectome quickDisco utility.

    Parameters
    ----------
    lesion_path : str
        Path to the lesion mask file (.nii or .nii.gz).
    output_path : str
        Target output path for the generated disconnectome.
    priors_h5_path : str
        Path to the HDF5 file containing the anatomical priors.
    n_proc : int
        Number of parallel processes to use (default=1).
    """
    # Generate the disconnectome map
    # This combines functional signals from distant voxels using anatomical priors.
    quickDisco.probaMap_fromMask(
        roiFile=lesion_path,
        priorsLoc=priors_h5_path,
        priors_type='h5',        # Set the priors storage format to HDF5.
        outFile=output_path,     # Directly save the resulting map.
        proc=n_proc,             # Assign parallel workers.
        maxVal=True              # Normalize the output map values between 0 and 1.
    )

    # Load and return the data for verification
    disconn_img = nib.load(output_path)
    return disconn_img.get_fdata()

In [8]:
# =============================================================================
# Test
# =============================================================================
import glob

lesion_files = sorted(glob.glob(os.path.join(LESIONS_DIR, "*.nii*")))
print(f"Total lesions: {len(lesion_files)}")

test_lesion = lesion_files[0]
test_output = os.path.join(DISCONN_DIR, "test_disconn.nii.gz")

print(f"Testing: {os.path.basename(test_lesion)}")

try:
    disconn = generate_disconnectome(test_lesion, test_output, PRIORS_H5, n_proc=2)
    print(f"Success! Shape: {disconn.shape}")
    print(f"Range: [{disconn.min():.4f}, {disconn.max():.4f}]")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

Total lesions: 4119
Testing: lesion0000_NA_NA.nii.gz
Success! Shape: (91, 109, 91)
Range: [0.0000, 1.0000]


In [7]:
# Note: Configure for fast batch processing
import os
import psutil

# 1. Detect Total RAM (in GB)
# Formula: Total Bytes / (1024^3)
ram_gb = round(psutil.virtual_memory().total / (1024**3))

# 2. Detect CPU Cores
# This identifies the number of virtual CPUs assigned to the instance
cpu_cores = os.cpu_count()

# 3. Categorization Logic
# If the instance has more than 20GB, it's a High-RAM environment
if ram_gb > 20:
    environment = "High-RAM (Pro)"
    # In High-RAM, we can use all available cores safely to maximize multiprocessing
    N_PROC = cpu_cores
else:
    environment = "Standard (Free/Pro Normal)"
    # In standard environments (~13GB), we limit N_PROC to 2 to avoid memory crashes
    N_PROC = min(2, cpu_cores)

print(f"--- Environment Configuration ---")
print(f"Instance type: {environment}")
print(f"Total RAM: {ram_gb} GB")
print(f"CPU Cores: {cpu_cores}")
print(f"Suggested N_PROC: {N_PROC}")
print(f"--------------------------------")

--- Environment Configuration ---
Instance type: High-RAM (Pro)
Total RAM: 47 GB
CPU Cores: 24
Suggested N_PROC: 24
--------------------------------


In [8]:
# =============================================================================
# Batch Processing (Execute Multiple Times)
# =============================================================================
import os
import glob
import pickle
from tqdm import tqdm

# Load previous progress to avoid reprocessing files
if os.path.exists(PROGRESS_FILE):
    with open(PROGRESS_FILE, 'rb') as f:
        processed = pickle.load(f)
    print(f"Resuming: {len(processed)} files already processed")
else:
    processed = set()

# Identify pending files
lesion_files = sorted(glob.glob(os.path.join(LESIONS_DIR, "*.nii*")))
pending = [f for f in lesion_files if os.path.basename(f) not in processed]
print(f"Pending: {len(pending)} / {len(lesion_files)}")

# Batch Configuration
# Note: Use N_PROC = 8 only if you have activated High-RAM in Colab Pro
BATCH_SIZE = 750
N_PROC = 16  # Parallel processes using multiprocessing pool (# Increased batch size because N_PROC=24 is very fast)

for lesion_path in tqdm(pending[:BATCH_SIZE], desc="Generating disconnectomes"):
    filename = os.path.basename(lesion_path)
    output_path = os.path.join(DISCONN_DIR, filename)

    if not os.path.exists(output_path):
        try:
            generate_disconnectome(lesion_path, output_path, PRIORS_H5, n_proc=N_PROC)
            processed.add(filename)
        except Exception as e:
            print(f"\nError processing {filename}: {e}")
    else:
        processed.add(filename)

    # Save progress to the pickle file every 50 iterations for safety
    if len(processed) % 250 == 0:
        with open(PROGRESS_FILE, 'wb') as f:
            pickle.dump(processed, f)

# Save final progress for the current batch execution
with open(PROGRESS_FILE, 'wb') as f:
    pickle.dump(processed, f)

print(f"\n{'='*50}")
print(f"Batch completed: {len(processed)} / {len(lesion_files)} total files")
print(f"Remaining files: {len(lesion_files) - len(processed)}")
print(f"{'='*50}")
print("Run the cell again for the next batch.")

Resuming: 3570 files already processed
Pending: 549 / 4119


Generating disconnectomes: 100%|██████████| 549/549 [1:19:06<00:00,  8.65s/it]


Batch completed: 4119 / 4119 total files
Remaining files: 0
Run the cell again for the next batch.


In [9]:
# =============================================================================
# Celda 7: Verificar resultados
# =============================================================================
disconn_files = sorted(glob.glob(os.path.join(DISCONN_DIR, "*.nii*")))
print(f"Disconnectomes generados: {len(disconn_files)}")

if disconn_files:
    sample = nib.load(disconn_files[0])
    lesion = nib.load(lesion_files[0])

    print(f"\nVerificación de shapes:")
    print(f"  Lesión: {lesion.shape}")
    print(f"  Disconn: {sample.shape}")
    print(f"  Match: {'Good' if lesion.shape == sample.shape else 'Bad'}")
    print(f"  Rango valores: [{sample.get_fdata().min():.4f}, {sample.get_fdata().max():.4f}]")

Disconnectomes generados: 4119

Verificación de shapes:
  Lesión: (91, 109, 91)
  Disconn: (91, 109, 91)
  Match: Good
  Rango valores: [0.0000, 1.0000]


#### Tiempo real estimado  

4,119 lesiones / 50 por batch = ~83 ejecuciones
....
Cada batch: ~25 min 50 por batch CPU (N_PROC = 2 o 4) -> 4 batch = 100 min

Cada batch: ~15 min 100 por batch CPU (N_PROC = 8) -> 2 batch = 30 min

Cada batch: ~45 min 300 por batch CPU (N_PROC = 16) -> 1  batch = 1200 min

Cada batch: ~75 min 500 por batch CPU (N_PROC = 16) -> 2  batch = 150 min

Cada batch: ~75 min 750 por batch CPU (N_PROC = 16) -> 2  batch = 150 min

Total: ~7-10 horas (ejecutando manualmente cada batch)